In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [ ]:
# Read dataset
use_cols = [
    'measured log(solubility:mol/L)', 'SMILES', 'Quality'
]
data = pd.read_csv('delaney.csv', usecols=use_cols)
print(data.shape)
print(data.head(10))

In [ ]:
# Check for missing data

for colname in use_cols:
    print('Column name - {}, Null values found - {}, Count - {}'.format(colname,data[colname].isnull().values.any(),data[colname].isnull().sum()))

data['Quality'].fillna(data['Quality'].value_counts().idxmax(), inplace=True)
data.head(10)

In [ ]:
# Convert SMILES to RDKit object
mol_list= []
for element in data.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

In [ ]:
# Calculate molecular descriptors from SMILES
'''
cLogP (Octanol-water partition coefficient)
MW (Molecular weight)
RB (Number of rotatable bonds)
AP (Aromatic proportion = number of aromatic atoms / number of heavy atoms)
'''

def generate_MolDes(smiles, verbose=False):
    '''
    Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation
                    https://towardsdatascience.com/how-to-use-machine-learning-for-drug-discovery-1ccb5fdf81ad#885b
     '''
    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)

        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["MolLogP","MolWt","NumRotatableBonds"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [ ]:
# Calculate cLogP,MW,RB
desc_df = generate_MolDes(data.SMILES)
# Calculate AP
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_df['Aromatic proportion'] = desc_AromaticProportion
print(desc_df.head(10))

In [ ]:
# Create X and Y
encoded_columns = pd.get_dummies(data['Quality'])
X = pd.concat([desc_df,encoded_columns], axis=1)
Y = data['measured log(solubility:mol/L)']

print(X.head(10))

In [ ]:
# Linear Regression Model Train and Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
Y_pred_train = model.predict(X_train)
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'% mean_squared_error(Y_train, Y_pred_train))